In [1]:
# ! pip install openpyxl xlsxwriter pandas tqdm requests

In [2]:
import requests, json, pandas as pd
from time import sleep
from tqdm import tqdm

# pandas show all columns 
pd.set_option('display.max_columns', None)

# pandas wider columns
pd.set_option('display.max_colwidth', None)

In [3]:
# !!! set to True if you want to update all data from the very beginning
full_update = False

https://politdata.nazk.gov.ua/api/swagger/#/reports/get_getreport__id_  
https://politdata.nazk.gov.ua/#/  
https://docs.google.com/document/d/1X68OvBYCQEiGG_nbjiD2fE0PZ1JIvbSsfXjrHUjOZ0c/edit?usp=sharing  

## Інформація про партії

In [4]:
# таблиця зі всіма партіями (api/getpartylistmain)

party_list = requests.get('https://politdata.nazk.gov.ua/api/getpartylistmain').json()
party_list = pd.DataFrame(party_list['data'])
party_list = party_list.drop(['locationFact','politPartyUnitId','locations'], axis=1)


# таблиця зі всіма регіональними представництвами (api/getpartylistregion)

party_region_list = requests.get('https://politdata.nazk.gov.ua/api/getpartylistregion').json()
party_region_list = pd.DataFrame(party_region_list['data'])
party_region_list = party_region_list.drop(['locationFact','locations'], axis=1) 

# додати інформацію про центральні партії до регіональних філій
party_region_list = party_region_list.merge(party_list[['name','unitId','code']]
                                            .rename({'name':'party_main_name', 'code':'party_main_EDRPOU','unitId':'party_main_unitId'}, 
                                                    axis=1), 
                                            right_on='party_main_unitId', 
                                            left_on='politPartyUnitId', 
                                            how='left')

# перейменувати
party_region_list = party_region_list.rename({'name':'local_org_name', 'code':'local_org_EDRPOU'},axis=1)

245


,name,unitId,code
0,ПОЛІТИЧНА ПАРТІЯ УКРАЇНСЬКА ПАРТІЯ «ЗЕЛЕНА ПЛАНЕТА»,101006351,33447027


## Download all reports

In [6]:
# завантажити список всіх звітів, які є в системі

report_list = requests.get('https://politdata.nazk.gov.ua/api/getreportslist').json()
report_list = [X['id'] for X in report_list]
len(report_list)

772

In [70]:
# завантажити список всіх звітів, які вже були завантажені

with open('data/data_for_downloader/downloaded_report_ids.txt', 'r') as f:
    downloaded_report_ids = f.read().splitlines()

len(downloaded_report_ids)

772

In [71]:
# у залежності від типу оновлення (повний чи частковий) визначити список звітів, які потрібно завантажити

if full_update:
    reports_to_download = report_list
else:
    reports_to_download = [x for x in report_list if x not in downloaded_report_ids]

len(reports_to_download)

0

In [9]:
r_df = pd.DataFrame()
for i in tqdm(reports_to_download):
    one_report = requests.get(f'https://politdata.nazk.gov.ua/api/getreport/{i}').json()
    
    # df
    one_report_df = pd.DataFrame(one_report)
    one_report_df['report_id'] = i
    r_df = pd.concat([r_df, one_report_df], axis=0)

    sleep(0.2)

r_df = r_df.reset_index(drop=True)

  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:01<00:00,  2.72it/s]


In [10]:
len(r_df)

12

In [11]:
if full_update:

    # створити новий файл зі всіма завантаженими звітами
    with open('data/data_for_downloader/downloaded_report_ids.txt', 'w') as f:
        for report_id in r_df['report_id'].unique():
            f.write(str(report_id) + '\n')

else:
    
    # додати підвантажені звіти до txt файлу
    with open('data/data_for_downloader/downloaded_report_ids.txt', 'a') as f:
        for report_id in r_df['report_id'].unique():
            f.write(str(report_id) + '\n')

In [12]:
# додати інфу про центральний офіс (який був визначений в кожному звіті)
party_main_finder = r_df.loc[r_df.officeType=="Центральний офіс",['report_id','partyCode','partyName']].rename({'partyCode':'party_main_EDRPOU','partyName':'party_main_name'}, axis=1)

# обʼєднати r_df та party_main_finder
r_df = r_df.merge(party_main_finder, on='report_id', how='left')

# перевірити помилки
len(r_df[r_df.party_main_name.isnull()])

0

In [13]:
# Чистка назв партій

r_df['party_main_name'] = r_df['party_main_name'].str.upper()

to_delete = ['^ПОЛІТИЧНА ПАРТІЯ ', '^ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ ', 'ВСЕУКРАЇНСЬКЕ ПОЛІТИЧНЕ ОБ\'ЄДНАННЯ',
             'ПОЛІТИЧЯНА ПАРТІЯ','СОЦІАЛЬНО-ЕКОЛОГІЧНА ПАРТІЯ', 'ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ', 'СОЦІАЛЬНО-ПОЛІТИЧНИЙ СОЮЗ ',
             '«', '»', '\"']
for words in to_delete:
    r_df['party_main_name'] = r_df['party_main_name'].str.replace(words, '', regex=True)

# прибрати зайві пробіли
r_df['party_main_name'] = r_df['party_main_name'].str.replace('\s+', ' ', regex=True).str.strip()


# ручні правки
party_renamer = {
    'РІШУЧИХ ДІЙ': 'ПАРТІЯ РІШУЧИХ ДІЙ',
    'КОНКРЕТНИХ СПРАВ':'ПАРТІЯ КОНКРЕТНИХ СПРАВ',
    'БЛОК СВІТЛИЧНОЇ РАЗОМ!':'БЛОК СВІТЛИЧНОЇ «РАЗОМ!»',
    'ПАРТІЯ ВОЛОДИМИРА БУРЯКА ЄДНАННЯ':'ПАРТІЯ ВОЛОДИМИРА БУРЯКА «ЄДНАННЯ»',
    'МИР':'ВСЕУКРАЇНСЬКЕ ОБ\'ЄДНАННЯ «МИР»',
    'ПАРТІЯ ІГОРЯ КОЛИХАЄВА НАМ ТУТ ЖИТИ!':'ПАРТІЯ ІГОРЯ КОЛИХАЄВА «НАМ ТУТ ЖИТИ!»',
    'КОМАНДА МАКСИМА ЄФІМОВА НАШ КРАМАТОРСЬК':'КОМАНДА МАКСИМА ЄФІМОВА «НАШ КРАМАТОРСЬК»',
    'ГРОМАДСЬКО-ПОЛІТИЧНИЙ РУХ ВАЛЕНТИНА НАЛИВАЙЧЕНКА СПРАВЕДЛИВІСТЬ':'ГРОМАДСЬКО-ПОЛІТИЧНИЙ РУХ ВАЛЕНТИНА НАЛИВАЙЧЕНКА «СПРАВЕДЛИВІСТЬ»',
    'МАЛОГО І СЕРЕДНЬОГО БІЗНЕСУ УКРАЇНИ':'ПАРТІЯ МАЛОГО І СЕРЕДНЬОГО БІЗНЕСУ УКРАЇНИ',
    'БЛОК ВІЛКУЛА УКРАЇНСЬКА ПЕРСПЕКТИВА':'БЛОК ВІЛКУЛА «УКРАЇНСЬКА ПЕРСПЕКТИВА»',
    'НАЦІОНАЛЬНО-ДЕМОКРАТИЧНЕ ОБ\'ЄДНАННЯ УКРАЇНА':'НАЦІОНАЛЬНО-ДЕМОКРАТИЧНЕ ОБ\'ЄДНАННЯ «УКРАЇНА»',
    'ГРОМАДЯНСЬКИЙ РУХ СВІДОМІ':'ГРОМАДЯНСЬКИЙ РУХ «СВІДОМІ»',
    'ВО ПЛАТФОРМА ГРОМАД':'ПЛАТФОРМА ГРОМАД'
}

r_df['party_main_name'] = r_df['party_main_name'].replace(party_renamer)

## Preparation

### Main functions

In [15]:
# прочитати всі renamers
with open('data/data_for_downloader/renamers.json', 'r') as f:
    renamers = json.load(f)

# зберігти кожен в окрему змінну
for k,v in renamers.items():
    exec(f"{k} = v")

In [16]:
def subset_table(df, main_var, cols_to_select):
    # subset
    table = df[[main_var] + cols_to_select]
    
    # видалити пусті
    table = table[table[main_var].apply(lambda x: len(x)>0)]
    
    return table

In [17]:
# замінити *** (деперсоналізовані дані) на None

def replace_stars(cell):
    if isinstance(cell, str) and (set(cell) == {'*'} or set(cell) == {'*','_'}):
        return None
    else:
        return cell

In [18]:
# основна функції розпавку змінної зі списком в індивідуальні рядки

def list_to_rows(table, main_var, cols_to_select, renamer):

    t = pd.DataFrame()

    for i in table.index:
        new_df = pd.DataFrame(table.loc[i, main_var])
        for n in cols_to_select:
            new_df[n] = table.loc[i,n]
        t = pd.concat([t, new_df], axis=0)

    # перейменувати
    t = t.rename(renamer, axis=1)
    t = t.reset_index(drop=True)
    
    # замінити ***
    t = t.applymap(replace_stars)

    return t

In [19]:
# зберегти як excel

def save_as_excel(table, filename, full_update):
    path = f'data/excel_tables/{filename}.xlsx'
    
    if full_update:
        table.to_excel(path, index=False, engine='xlsxwriter')
    
    else:
        # open and save in excel_df
        excel_df = pd.read_excel(path)
        if len(table) > 0:
            new_table = pd.concat([excel_df, table], axis=0, ignore_index=True)
            new_table.to_excel(path, index=False, engine='xlsxwriter')
        else:
            excel_df.to_excel(path, index=False, engine='xlsxwriter')

### Functions for data cleaning

In [20]:
# почистити IBAN
def clean_bank_account(bank_account_column):
    clean_bank_account_column = (bank_account_column.str.replace('№','', regex=True)
                                .str.replace(' ','', regex=True)
                                .str.replace(':','', regex=True)
                                .str.replace('\n','', regex=True)
                                .str.strip())
    return clean_bank_account_column

In [21]:
# Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінити на “партійний осередок” (визначила по 'donor_edrpou')

def check_edrpou_for_party(table, var_edrpou_to_search, var_to_replace):
    table.loc[(table[var_edrpou_to_search].isin(party_list.code.tolist()))|
              (table[var_edrpou_to_search].isin(party_region_list.local_org_EDRPOU.tolist())), 
              var_to_replace] = 'Партійний осередок'
    return table

In [22]:
org_names_to_rename = {
 'ТОВ ПІДПРИЄМСТВО КИЇВ': 'ТОВ ПІДПРИЄМСТВО "КИЇВ"',
 'ТОВ "ПІДПРИЄМСТВО "КИЇВ"': 'ТОВ ПІДПРИЄМСТВО "КИЇВ"',
 'ПІДПРИЄМСТВО "КИЇВ" ТОВ': 'ТОВ ПІДПРИЄМСТВО "КИЇВ"',
 'ТОВ "ПІДПРИЄМСТВО КИЇВ"': 'ТОВ ПІДПРИЄМСТВО "КИЇВ"',
 'ПІДПРИЄМСТВО "КИЇВ"': 'ТОВ ПІДПРИЄМСТВО "КИЇВ"',
 'АКЦІОНЕРНЕ ТОВАРИСТВО "РАЙФФАЙЗЕН БАНК"': 'АКЦІОНЕРНЕ ТОВАРИСТВО "РАЙФФАЙЗЕН БАНК АВАЛЬ"'}

In [23]:
def org_name_clean(var_name):

    # manual replacers
    t[var_name] = t[var_name].replace(org_names_to_rename)

    # replace 'ФІЗИЧНА ОСОБА ПІДПРИЄМЕЦЬ' with 'ФОП' inside each cell

    fop_replacers = ['ФІЗИЧНА ОСОБА ПІДРИЄМЕЦЬ', 'ФІЗИЧНА ОСОБА ПІДПРИЄМЕЦЬ','ФІЗИЧНА ОСОБА ПІДПРИЄМЕЦЬ',
                    'ФІЗИЧНА ОСОБА- ПІДПРИЄМЕЦЬ','ФІЗИЧНА ОСОБА-ПІДПРИЄМЕЦЬ','ФІЗИЧНА ОСОБА-ПІДРИЄМЕЦЬ','ФІЗИЧНА ОСОБА - ПІДПРИЄМЕЦЬ',
                    'ФІЗИЧНА ОСОБА -ПІДПРИЄМЕЦЬ','ФІЗИЧНА ОБОБА-ПІДПРИЄМЕЦЬ','ФІЗИЧНА ОСОБА-ПІДПРИМЕЦЬ']
    for replacer in fop_replacers:
        t[var_name] = t[var_name].str.replace(replacer,'ФОП', regex=True)

    # Long to short form of ТОВ
    tov_replacers = ["ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПОВІДАЛЬНІСТЮ","ТОВАРИСТВО З ОБМЕЖЕНОЮ ВІДПРОВІДАЛЬНІСТЮ"]
    for replacer in tov_replacers:
        t[var_name] = t[var_name].str.replace(replacer,'ТОВ', regex=True)

    t[var_name] = t[var_name].str.replace("ПРИВАТНЕ АКЦІОНЕРНЕ ТОВАРИСТВО",'ПрАТ', regex=True)
    
    for replacer in ["ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО", 'ПУБЛІЧНЕ АКЦІОНЕРНЕ ТОВАРИСТВО']:
        t[var_name] = t[var_name].str.replace(replacer,'ПАТ', regex=True)

    for replacer in ["ДЕРЖАВНОЇ ПОДАТКОВОЇ СЛУЖБИ", 'ДЕРЖАВНА ПОДАТКОВА СЛУЖБА']:
        t[var_name] = t[var_name].str.replace(replacer,'ДПС', regex=True)
    

    t[var_name] = t[var_name].str.replace('ПОЛІТИЧНА ПАРТІЯ','ПП', regex=True)

    for replacer in ["АКЦІОНЕРНЕ ТОВАРИВСТВО",'АКЦІОНЕРНЕ ТОВАРИСТВО']:
        t[var_name] = t[var_name].str.replace(replacer,'АТ', regex=True)    

    # delete 'ФОП' at the end of the string and put at the start
    for i in t.index:
        if type(t.loc[i,var_name]) == str: 
            if t.loc[i,var_name].endswith('ФОП'):
                t.loc[i,var_name] = 'ФОП ' + t.loc[i,var_name].replace(' ФОП','').strip() 


## Tables

### Таблиця 1: загальна інформація про партію/осередок

In [24]:
filename = '1_legal_entity_report_info'

In [25]:
cols_to_select = [x for x in renamer_1.keys() if x in r_df.columns]
table = r_df[cols_to_select].rename(renamer_1, axis=1)

# add None if empty column
for v in renamer_1.values():
    if v not in table.columns:
        table[v] = None

table = table[list(renamer_1.values())]

table.head(1)

,officeType,report_submition_date,documentId,report_type,report_period,report_year,legal_entity_name,legal_entity_edrpou,legal_entity_country,legal_entity_index,legal_entity_region,legal_entity_district,legal_entity_city,legal_entity_street,legal_entity_building,legal_entity_korpus,legal_entity_apartment,is_actual_address,actual_address_country,actual_address_index,actual_address_region,actual_address_district,actual_address_city,actual_address_street,actual_address_building,actual_address_korpus,actual_address_apartment,legal_entity_head_last_name,legal_entity_head_first_name,legal_entity_head_middle_name,number_of_employees_contract,number_of_employees_civil_agreement,number_of_employees_volunteers,report_id,party_main_name,party_main_EDRPOU
0,Центральний офіс,2024-04-23T06:29:13.477Z,bab43d40-013a-11ef-8a6c-27bc4724684c,Основний,рік,2021,ПОЛІТИЧНА ПАРТІЯ «АДВОКАТ»,40124783,Україна,03191,м.Київ,Голосіївський,NaN,вул. Лятошинського композитора,14,А,NaN,Так,Україна,None,None,None,None,None,None,None,None,Стародуб,Юлія,Валентинівна,None,None,None,ce536ec0-013a-11ef-938a-5514903835ea,АДВОКАТ,40124783


In [26]:
# add report about duplicates
table[table.duplicated(subset=['report_period','report_year','legal_entity_edrpou',],keep=False)]

,officeType,report_submition_date,documentId,report_type,report_period,report_year,legal_entity_name,legal_entity_edrpou,legal_entity_country,legal_entity_index,legal_entity_region,legal_entity_district,legal_entity_city,legal_entity_street,legal_entity_building,legal_entity_korpus,legal_entity_apartment,is_actual_address,actual_address_country,actual_address_index,actual_address_region,actual_address_district,actual_address_city,actual_address_street,actual_address_building,actual_address_korpus,actual_address_apartment,legal_entity_head_last_name,legal_entity_head_first_name,legal_entity_head_middle_name,number_of_employees_contract,number_of_employees_civil_agreement,number_of_employees_volunteers,report_id,party_main_name,party_main_EDRPOU


In [27]:
save_as_excel(table, filename, full_update)

/var/folders/7x/nddt8r3s795cdr4ndj4l1tl00000gn/T/ipykernel_56305/4081751323.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_table = pd.concat([excel_df, table], axis=0, ignore_index=True)


### Таблиця 2: інформація про осередки політичної партії  
Назва файлу: 2.2_other_party_orgs_info

In [28]:
main_var = 'tablets1'

cols_to_select = ['partyLocationRegion','period', 'year', 'party_main_name','party_main_EDRPOU','report_id', 'officeType']

filename = '2.1_local_orgs_info'

In [29]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_2)
    t = t[list(renamer_2.values())+['party_main_name','party_main_EDRPOU','report_id','officeType']]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'
    # і потім видалити 'officeType'
    t = t.drop('officeType', axis=1)

    save_as_excel(t, filename, full_update)

/var/folders/7x/nddt8r3s795cdr4ndj4l1tl00000gn/T/ipykernel_56305/1600581565.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t = t.applymap(replace_stars)


### Таблиця 3: інформація про інші підприємства, установи та організації створені партією політичної партії

In [30]:
cols_to_select = ['period', 'year', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'tablets2'

filename = '2.2_other_party_orgs_info'

In [31]:
table = subset_table(r_df, main_var, cols_to_select)
len(table)

0

In [32]:
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_3)
    t = t[list(renamer_3.values())+['party_main_name','party_main_EDRPOU','report_id']]
    
    # delete empty values
    t = t[(t.other_party_org_name != '0')|(t.other_party_org_EDRPOU != '00000000')]

    save_as_excel(t, filename, full_update)

### Таблиця 4: зведена інформація про нерухоме майно політичної партії (propertyObjects)

In [33]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode','partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyObjects'

filename = '3.1_property_objects'

In [34]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_4)
    t = t[list(renamer_4.values())+['party_main_name','party_main_EDRPOU','report_id']]

    # delete empty
    t = t[t.object_type.notna()]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['object_owner_name']

    if len(t.object_owner_name.unique()) > 1:

        t['object_owner_name'] = t['object_owner_name'].str.upper()
        t['object_owner_name'] = t['object_owner_name'].str.replace('\s+',' ', regex=True).str.strip()

        # уніфікувати ФОП, ТОВ і ПАТ
        org_name_clean('object_owner_name')

        # уніфікувати апострофи в іменах
        t['object_owner_name'] = t['object_owner_name'].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

        # replace ocassional english letters
        t.loc[t['object_owner_name'].str.contains('[А-Я]', na=False),'object_owner_name'] = t.loc[t['object_owner_name'].str.contains('[А-Я]', na=False),'object_owner_name'].str.replace('C','С').str.replace('I','І')

        save_as_excel(t, filename, full_update)

### Таблиця 5: зведена інформація про рухоме майно політичної партії (propertyMovables)

In [35]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyMovables'

filename = '3.2_movable_property'

In [36]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_5)
    
    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # sort columns
    sorter = [x for x in list(renamer_5.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
    t = t[sorter]

    # clean t['object_owner_name']
    var_to_clean = 'object_owner'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


### Таблиця 6: зведена інформація про автотранспорт політичної партії (propertyTransport)

In [37]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyTransport'

filename = '3.3_vehicles'

In [38]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_6)

    # sort columns
    sorter = [x for x in list(renamer_6.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
    t = t[sorter]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # delete empty
    t = t[(t.object_type.notna())&(t.object_brand.notna())]

    # Якщо object_owner_edrpou це код партії чи осередку то у object_owner_type ставити “Партійний осередок”
    t = check_edrpou_for_party(t, 'object_owner_edrpou', 'object_owner_type')

    # clean t['object_owner']
    var_to_clean = 'object_owner'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


### Таблиця 7: зведена інформація про цінні папери політичної партії (propertyPapers)

In [39]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyPapers'

filename = '3.4_securities'

In [40]:
table = subset_table(r_df, main_var, cols_to_select)
len(table)

0

In [41]:
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_7)

    if len(t) > 0:
        # sort columns
        sorter = [x for x in list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
        t = t[sorter]

        # Для центральних офісів політичних партій legal_entity_region –> “Україна”
        t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

        # save
        save_as_excel(t, filename, full_update)


    elif len(t) == 0:
        # create columns if t is empty
        t = pd.DataFrame(columns=list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id'])
        save_as_excel(t, filename, full_update)

else:
    # create columns if table is empty
    t = pd.DataFrame(columns=list(renamer_7.values())+['party_main_name','party_main_EDRPOU','report_id'])
    save_as_excel(t, filename, full_update)
    
    
    

### Таблиця 8: зведена інформація про інше майно та нематеріальні активи політичної партії (propertyNoMoney)

In [42]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyNoMoney'

filename = '3.5_intangible_assets'

In [43]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_8)

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    save_as_excel(t, filename, full_update)

### Таблиця 9: зведена інформація про банківські рахунки політичної партії (propertyMoney)

In [44]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'propertyMoney'

filename = '4_bank_accounts'

In [45]:
table = subset_table(r_df, main_var, cols_to_select)

if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_9)

    t['account_number'] = clean_bank_account(t['account_number'])

    # check if any are longer than 29
    t['account_number'].apply(lambda x: len(x)).value_counts()

    # t[t.account_number.apply(lambda x: len(x)>29)]

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    save_as_excel(t, filename, full_update)


/var/folders/7x/nddt8r3s795cdr4ndj4l1tl00000gn/T/ipykernel_56305/1600581565.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t = t.applymap(replace_stars)


### Таблиця 10: зведена інформація про приватні грошові внески на рахунки політичної партії (contributionConMoney)  

In [46]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU',
                  'partyLocationRegion','report_id']

main_var = 'contributionConMoney'

filename = '5_private_contributions'

In [47]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_10)

    # delete empty
    t = t[(t.donor_name.notna())&(t.bank_edrpou.notna())]

    # clean bank_account
    t['bank_account'] = clean_bank_account(t['bank_account'])

    # Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінити на “партійний осередок” (визначила по 'donor_edrpou')
    t = check_edrpou_for_party(t, 'donor_edrpou', 'donor_type')

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # sort columns
    t = t[list(renamer_10.values())+['party_main_name','party_main_EDRPOU','report_id']]

    # clean t['donor_name']
    var_to_clean = 'donor_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 11: зведена інформація про негрошові внески на користь політичної партії (contributionOtherCon)

In [48]:

cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','partyLocationRegion','report_id']

main_var = 'contributionOtherCon'

filename = '6_in_kind_donations'

In [49]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_11)

    # delete empty
    t = t[(t.donor_name.notna())&(t.donor_type.notna())]

    # Якщо серед донорів зустрічаються партійні осередки чи центральний офіс політичної партії 'donor_type' замінила на “партійний осередок” (визначила по 'donor_edrpou')

    t.loc[(t.donor_edrpou.isin(party_list.code.tolist()))|
        (t.donor_edrpou.isin(party_region_list.local_org_EDRPOU.tolist())), 
        'donor_type'] = 'Партійний осередок'

    # donor_edrpou delete *
    t.loc[t.donor_edrpou.str.contains('\*',na=False),'donor_edrpou'] = None
    t.loc[t.donor_birth_date.str.contains('\*',na=False),'donor_birth_date'] = None
    t.loc[t.object_registration_number.str.contains('\*',na=False),'object_registration_number'] = None

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”

    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['donor_name']
    var_to_clean = 'donor_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 12: зведена інформація про кошти державного фінансування виплачені політичним партіям (contributionCosts)
Завдання: об'єднати всі дані щодо коштів державного фінансування отриманих
партіями. Також, потрібно почистити costs3/bank_account (пробіли, зайві символи).  
  
Назва файлу: 7_state_funding_transactions

In [50]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'contributionCosts'

filename = '7_state_funding_transactions'

In [51]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_12)

    # convert refund_sum to numeric
    t['transaction_sum'] = t['transaction_sum'].astype(int)

    t['bank_account'] = clean_bank_account(t['bank_account'])

    # sort columns
    t = t[list(renamer_12.values()) + ['party_main_name','party_main_EDRPOU','report_id']]

    save_as_excel(t, filename, full_update)

### Таблиця 13: зведена інформація про інші грошові надходження на рахунки політичних партій (contributionOtherCosts)

In [52]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'contributionOtherCosts'

filename = '8_other_income'

In [53]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_13)
    t['bank_account'] = clean_bank_account(t['bank_account'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['sender_name']
    var_to_clean = 'sender_name'

    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 14: зведена інформація про витрати державного фінансування політичних партій (paymentGov)

In [54]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU',
                  'partyLocationRegion','report_id']

main_var = 'paymentGov'

filename = '9.1_expenditures_public_funding'

In [55]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_14)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # Якщо отримувач коштів є партійною структурою (recipient_EDRPOU), у recipient_type замінити позначку “Юридична особа/Фізична особа” на Партійний осередок.
    t = check_edrpou_for_party(t, 'recipient_EDRPOU', 'recipient_type')

    # clean t['recipient_name']
    var_to_clean = 'recipient_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### Таблиця 15: зведена інформація про витрати з рахунків з приватним фінансуванням (paymentOther)

In [56]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','partyLocationRegion','report_id']

main_var = 'paymentOther'

filename = '9.2_expenditures_private_funds'

In [57]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_15)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # Якщо отримувач коштів є партійною структурою (recipient_EDRPOU), у recipient_type замінити позначку “Юридична особа/Фізична особа” на Партійний осередок.
    t = check_edrpou_for_party(t, 'recipient_EDRPOU', 'recipient_type')

    # delete empty
    t = t[(t.bank_EDRPOU.notna())&(t.account_number.notna())]

    # clean t['recipient_name']
    var_to_clean = 'recipient_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

/var/folders/7x/nddt8r3s795cdr4ndj4l1tl00000gn/T/ipykernel_56305/1600581565.py:16: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  t = t.applymap(replace_stars)


### Таблиця 16: зведена інформація про отримання помилкових надходжень (paymentCostsPaymentReceive)

In [58]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentCostsPaymentReceive'

filename = '9.3_false_donations_info'

In [59]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:

    t = list_to_rows(table, main_var, cols_to_select, renamer_16)

    t['account_number'] = clean_bank_account(t['account_number'])

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # clean t['donor_name']
    var_to_clean = 'donor_name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)

### НЕ ІСНУЄ Таблиця 17: інформація про повернення внесків та помилкових надходжень (paymentCostsPaymentReturn)
Завдання: обєднати всю інформацію про повернення помилкових внесків.  
  
Назва файлу: 9.4_false_donations_returning

In [60]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentCostsPaymentReturn'

filename = '9.4_false_donations_returning'

In [61]:
main_var in r_df.columns

False

### Таблиця 18: інформація про отримання помилкових інших (негрошових) внесків (paymentOtherCostsPaymentReceive)

In [62]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion', 
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentOtherCostsPaymentReceive'

filename = '9.5_false_in_kind_donations_info'

In [63]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_18)

    if len(t) == 0:
        # create columns if t is empty
        t = pd.DataFrame(columns=list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id'])
        save_as_excel(t, filename, full_update)

    else:
        # sort columns
        sorter = [x for x in list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id']  if x in t.columns]
        t = t[sorter]

        # Для центральних офісів політичних партій legal_entity_region –> “Україна”
        t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

        # clean t['recipient_name']

        var_to_clean = 'onor_name'

        t[var_to_clean] = t[var_to_clean].str.upper()
        t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

        # уніфікувати ФОП, ТОВ і ПАТ
        org_name_clean(var_to_clean)

        # уніфікувати апострофи в іменах
        t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

        # replace ocassional english letters
        t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

        # save
        save_as_excel(t, filename, full_update)

else:
    # create t
    t = pd.DataFrame(columns=list(renamer_18.values())+['party_main_name','party_main_EDRPOU','report_id'])
    save_as_excel(t, filename, full_update)
        

### НЕ ІСНУЄ Таблиця 19: інформація про отримання помилкових інших (негрошових) внесків (paymentOtherCostsPaymentReturn)

In [64]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'party_main_name','party_main_EDRPOU','report_id']

main_var = 'paymentOtherCostsPaymentReturn'

filename = '9.5_false_in_kind_donations_returning'

In [65]:
main_var in r_df.columns

False

### Таблиця 20: інформація про фінансові зобов'язання (obligate)

In [66]:
cols_to_select = ['officeType','types','period', 'year','partyName','partyCode', 'partyLocationRegion',
                  'party_main_name','party_main_EDRPOU','report_id']

main_var = 'obligate'

filename = '10_liabilities'

In [67]:
table = subset_table(r_df, main_var, cols_to_select)
if len(table) > 0:
    t = list_to_rows(table, main_var, cols_to_select, renamer_20)

    # Для центральних офісів політичних партій legal_entity_region –> “Україна”
    t.loc[t.officeType == 'Центральний офіс', 'legal_entity_region'] = 'Україна'

    # delete empty
    t = t[t.name.notnull()]

    # clean t['name']
    var_to_clean = 'name'
    t[var_to_clean] = t[var_to_clean].str.upper()
    t[var_to_clean] = t[var_to_clean].str.replace('\s+',' ', regex=True).str.strip()

    # уніфікувати ФОП, ТОВ і ПАТ
    org_name_clean(var_to_clean)

    # уніфікувати апострофи в іменах
    t[var_to_clean] = t[var_to_clean].replace(r"(?<=\w)[’\"\`](?=\w)", "\'", regex=True)

    # replace ocassional english letters
    t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean] = t.loc[t[var_to_clean].str.contains('[А-Я]', na=False),var_to_clean].str.replace('C','С').str.replace('I','І')

    save_as_excel(t, filename, full_update)


In [68]:
print('Done')

Done
